# TODO: this should become tests!

In [ ]:
# Third-party
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Custom
from matplotlib import cm
from scipy.optimize import minimize
from scipy.special import logsumexp
from sklearn.mixture import GaussianMixture

from density import get_projected_coords, get_u_v, get_uniform_idx, GaussianNoodle2D
from likelihood import z_to_a, a_to_z, StreamDensityModel

In [ ]:
rnd = np.random.RandomState(22)

h = 1.
true_K = 3
true_nodes = np.stack((np.linspace(0, 10, true_K),
                       np.zeros(true_K))).T
true_s = rnd.uniform(0.2, 0.5, size=true_K)

true_z = rnd.uniform(0.8, 0.95, size=true_K-1)
true_m = np.zeros(true_K)
true_a = z_to_a(true_z)
assert np.isclose(true_a.sum(), 1.)

In [ ]:
# Sample from truth:
C = np.zeros((true_K, 2, 2))
C[:, 0, 0] = h**2
C[:, 1, 1] = true_s**2

P = np.zeros_like(C)
for k in range(true_K):
    P[k] = np.linalg.cholesky(np.linalg.inv(C[k]))

gmm = GaussianMixture(n_components=true_K, covariance_type='full')
gmm.fit(np.random.random(size=(true_K+1, 2)))
gmm.weights_ = true_a
gmm.covariances_ = C
gmm.precisions_cholesky_ = P
gmm.means_ = true_nodes

# now add a uniform background
n_samples_total = 10000
true_f = 0.85
n_stream = int(true_f * n_samples_total)
gmm_X = gmm.sample(n_samples=n_stream)[0]

window_bounds = [[-5, 15], [-5, 5]]
n_bg = n_samples_total - n_stream
unif_X = np.stack([np.random.uniform(b[0], b[1], size=n_bg) 
                   for b in window_bounds]).T
X = np.vstack((gmm_X, unif_X))

In [ ]:
true_dens_model = GaussianNoodle2D(X, poly=np.poly1d([0.]))
track = true_dens_model.get_dense_poly_track(size=10000)
nodes = true_dens_model.set_nodes(track=track, nodes=true_nodes)

In [ ]:
def bg_ln_like(p, X):
    N = len(X)
    return np.full(N, np.sum([-np.log(b[1]-b[0]) for b in window_bounds]))

In [ ]:
model = StreamDensityModel(X, true_dens_model, h=h, 
                           bg_ln_likelihood=bg_ln_like, 
                           m_prior_sigma=0.1)
# frozen={'ln_s': np.log(true_s),
#         'ln_z': np.log(true_z),
#         'm': true_m}

In [ ]:
p0 = {'ln_s': np.log(true_s),
      'ln_z': np.log(true_z),
      'm': true_m,
      'f': true_f}

In [ ]:
model.ln_prior(p0)

In [ ]:
derivs, signs = model.ln_d_likelihood_dp(p0)

In [ ]:
# for name in derivs:
#     print(name, derivs[name].shape)

In [ ]:
full_derivs = model.d_ln_likelihood_dp(p0)

In [ ]:
# for name in full_derivs:
#     print(name, full_derivs[name].shape)

In [ ]:
def func_helper(x):
    p = model.unpack_pars(x)
    return model.ln_likelihood(p).sum()

def num_deriv_helper(x):
    eps = 1e-10
    num_derivs = []
    for k in range(x.size):
        x1 = x.copy()
        x2 = x1.copy()
        x2[k] = x2[k] + eps
        _d = (func_helper(x2) - func_helper(x1)) / eps
        num_derivs.append(_d)
    return np.array(num_derivs)

def deriv_helper(x):
    p = model.unpack_pars(x)
    derivs = model.d_ln_likelihood_dp(p)
    return np.concatenate(list(derivs.values()))

In [ ]:
x0 = model.pack_pars(**p0)
true_x = x0

In [ ]:
func_helper(x0)

In [ ]:
deriv_helper(x0)

In [ ]:
num_deriv_helper(x0)

In [ ]:
jj = 0
vals = np.linspace(0.1, 1-1e-3, 128)
# vals = np.linspace(np.log(0.1), np.log(5), 128)
lls = []
for val in vals:
    xx = x0.copy()
    xx[jj] = val
    lls.append(func_helper(xx))
lls = np.array(lls)
plt.plot(vals, np.exp(lls - lls.max()))
plt.axvline(true_x[jj])